# Reto 1 - Orientado a un Proyecto

# Contentido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Visión general de los datos](#data_review)
    * [Conclusiones](#data_review_conclusions)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores faltantes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)

## Introducción <a id='intro'></a>
Cuando estamos investigando, necesitamos formular hipótesis que luego podemos poner a prueba. A veces aceptamos estas hipótesis; otras veces las rechazamos. Para tomar decisiones acertadas, una empresa debe poder comprender si está haciendo las suposiciones correctas.

En este proyecto, compararás las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiarás datos reales de Yandex.Music para probar las siguientes hipótesis y comparar el comportamiento de los usuarios en estas dos ciudades.

### Objetivo: 
Poner a prueba tres hipótesis:
1. La actividad de los usuarios difiere según el día de la semana y la ciudad.
2. Los residentes de Springfield y Shelbyville escuchan géneros musicales diferentes los lunes por la mañana. Esto también es cierto para las tardes de los viernes.
3. Los oyentes de Springfield y Shelbyville tienen preferencias diferentes. En Springfield prefieren el pop, mientras que en Shelbyville hay más fanáticos del rap.

### Etapas 
Los datos sobre el comportamiento de los usuarios se encuentran en el archivo `/data/music.csv`. No hay información sobre la calidad de los datos, por lo que deberás explorarlos antes de probar las hipótesis.

Primero, evaluarás la calidad de los datos y verás si los problemas son significativos. Luego, durante el preprocesamiento de datos, intentarás solucionar los problemas más críticos.

Tu proyecto constará de tres etapas:
1. Visión general de los datos
2. Preprocesamiento de datos
3. Pruebas de las hipótesis

[Tabla de Contenido](#back)

## Etapa 1. Visión general de los datos <a id='data_review'></a>

Importar los paquetes necesarios:

Lee el archivo `music_project_en.csv` desde la carpeta `/datasets/` y guárdalo en la variable `df`:


Obtener información sobre los 10 primeros registros:

Obteniendo información general sobre los datos en df

La tabla contiene siete columnas. Todas ellas almacenan el mismo tipo de datos: `object`.

Según la documentación:
- `'userID'` — identificador de usuario
- `'Track'` — título de la canción
- `'artist'` — nombre del artista
- `'genre'` — género musical
- `'City'` — ciudad del usuario
- `'time'` — hora exacta en que se reprodujo la canción
- `'Day'` — día de la semana

Podemos observar tres problemas con el estilo de los nombres de las columnas:
1. Algunos nombres están en mayúsculas y otros en minúsculas.
2. Hay espacios en algunos nombres.
3. `Detecta el tercer problema tú mismo y descríbelo aquí`.



### Conclusiones <a id='data_review_conclusions'></a> 


Para avanzar, necesitamos preprocesar los datos.

[Tabla de Contenido](#back)

## Etapa 2. Pre-Procesamiento de Datos <a id='data_preprocessing'></a>
Corrige el formato en los encabezados de las columnas y ocúpate de los valores faltantes. Luego, verifica si hay duplicados en los datos.

### Estilo del Encabezado <a id='header_style'></a>
Imprime los headers del dataframe.

Cambiar los nombres de columna de acuerdo a las reglas de estilo adecuado:

    Si el nombre tiene varias palabras, utilizar snake_case
    Todos los caracteres deben estar en minúscula
    Eliminar espacios

Verifica el resultado. Imprime los nombres de las columnas una vez más:

[Back to Contents](#back)

### Valores Nulos <a id='missing_values'></a>
Primero, encuentra el número de valores faltantes en la tabla. Para hacerlo, utiliza dos métodos de pandas encadenados:

No todos los valores faltantes afectan la investigación. Por ejemplo, los valores faltantes en `track` y `artist` no son críticos. Simplemente puedes reemplazarlos con marcadores claros.

Sin embargo, los valores faltantes en `'genre'` pueden afectar la comparación de las preferencias musicales en Springfield y Shelbyville. En la vida real, sería útil conocer las razones por las cuales faltan los datos e intentar compensarlos. Pero no tenemos esa oportunidad en este proyecto. Por lo tanto, deberás:
* Rellenar estos valores faltantes con marcadores
* Evaluar cuánto pueden afectar los valores faltantes a tus cálculos


Reemplaza los valores faltantes en `'track'`, `'artist'` y `'genre'` con la cadena de texto `'unknown'`. Para hacer esto, crea la lista `columns_to_replace`, recórrela con un bucle `for` y reemplaza los valores faltantes en cada una de las columnas:


Asegúrate de que la tabla no contenga más valores faltantes. Vuelve a contar los valores faltantes.

[Tabla de Contenido](#back)

### Duplicados <a id='duplicates'></a>
Encuentra el número de duplicados obvios en la tabla utilizando dos comandos encadenados:

Llama al método de pandas para eliminar los duplicados obvios:

Vuelve a contar los duplicados obvios para asegurarte de haber eliminado todos ellos:

Ahora elimina los duplicados implícitos en la columna `'genre'`. Por ejemplo, el nombre de un género puede estar escrito de diferentes maneras. Estos errores también afectarán el resultado.

Imprime una lista de nombres de género únicos, ordenados alfabéticamente. Para hacerlo:

- Obtén la columna deseada del DataFrame
- Aplica un método de ordenamiento a esa columna
- Para la columna ordenada, llama al método que devolverá todos los valores únicos de la columna

Revisa la lista para encontrar duplicados implícitos del género `hiphop`. Estos podrían ser nombres escritos incorrectamente o nombres alternativos del mismo género.

Verás los siguientes duplicados implícitos:
* `hip`
* `hop`
* `hip-hop`

Para deshacerte de ellos, declara la función `replace_wrong_genres()` con dos parámetros:
* `wrong_genres=`: la lista de duplicados
* `correct_genre=`: la cadena con el valor correcto

La función debe corregir los nombres en la columna `'genre'` de la tabla `df`, es decir, reemplazar cada valor de la lista `wrong_genres` con el valor en `correct_genre`.


Llama a `replace_wrong_genres()` y pásale argumentos para que elimine los duplicados implícitos (`hip`, `hop` y `hip-hop`) y los reemplace con `hiphop`:


Asegúrate de que los nombres duplicados hayan sido eliminados. Imprime la lista de valores únicos de la columna `'genre'`:


[Tabla de Contenido](#back)

### Conclusiones <a id='data_preprocessing_conclusions'></a>

Hemos detectado tres problemas con los datos:

- Estilo incorrecto de encabezados
- Valores faltantes
- Duplicados obvios e implícitos

Los encabezados han sido corregidos para facilitar el procesamiento de la tabla.

Todos los valores faltantes han sido reemplazados por `'unknown'`. Sin embargo, aún debemos ver si los valores faltantes en `'genre'` afectarán nuestros cálculos.

La ausencia de duplicados hará que los resultados sean más precisos y más fáciles de entender.

Ahora podemos pasar a probar las hipótesis.


[Tabla de Contenidos](#back)

### Pruebas de las hipótesis

Escribe en esta sección que procedimiento seguirias para verificar estos supuestos. No es necesario usar código, sólo describir tu procedimiento.